# Assignment 1: Preprocessing and Text Classification

Student Name: Jeanelle Abanto

Student ID: 1133815

# General Info

<b>Due date</b>: Sunday, 28 March 2021 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 9% of mark for class (with 8% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/121115/pages/using-jupyter-notebook-and-python?module_item_id=2681264) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

# Overview

In this homework, you'll be working with a collection tweets. The task is to predict the geolocation (country) where the tweet comes from. This homework involves writing code to preprocess data and perform text classification.

# Preprocessing (4 marks)

**Instructions**: Download the data (as1-data.json) from Canvas and put it in the same directory as this iPython notebook. Run the code below to load the json data. This produces two objects, `x` and `y`, which contains a list of  tweets and corresponding country labels (it uses the standard [2 letter country code](https://www.iban.com/country-codes)) respectively. **No implementation is needed.**

In [1]:
import json

x = []
y = []
data = json.load(open("as1-data.json"))
for k, v in data.items():
    x.append(k)
    y.append(v)
    
print("Number of tweets =", len(x))
print("Number of labels =", len(y))
print("\nSamples of data:")
for i in range(10):
    print("Country =", y[i], "\tTweet =", x[i])
    
assert(len(x) == 943)
assert(len(y) == 943)

Number of tweets = 943
Number of labels = 943

Samples of data:
Country = us 	Tweet = @Addictd2Success thx u for following
Country = us 	Tweet = Let's just say, if I were to ever switch teams, Khalesi would be top of the list. #girlcrush
Country = ph 	Tweet = Taemin jonghyun!!! Your birits make me go~ http://t.co/le8z3dntlA
Country = id 	Tweet = depart.senior 👻 rapat perdana (with Nyayu, Anita, and 8 others at Ruang Aescullap FK Unsri Madang) — https://t.co/swRALlNkrQ
Country = ph 	Tweet = Done with internship with this pretty little lady!  (@ Metropolitan Medical Center w/ 3 others) [pic]: http://t.co/1qH61R1t5r
Country = gb 	Tweet = Wow just Boruc's clanger! Haha Sunday League stuff that, Giroud couldn't believe his luck! #clown
Country = my 	Tweet = I'm at Sushi Zanmai (Petaling Jaya, Selangor) w/ 5 others http://t.co/bcNobykZ
Country = us 	Tweet = Mega Fest!!!! Its going down🙏🙌  @BishopJakes
Country = gb 	Tweet = @EllexxxPharrell wow love the pic babe xx
Country = us 	Tweet = You 

### Question 1 (1.0 mark)

**Instructions**: Next we need to preprocess the collected tweets to create a bag-of-words representation. The preprocessing steps required here are: (1) tokenize each tweet into individual word tokens (using NLTK `TweetTokenizer`); (2) lowercase all words; (3) remove any word that does not contain any English alphabets (e.g. {_hello_, _#okay_, _abc123_} would be kept, but not {_123_, _!!_}) and (4) remove stopwords (based on NLTK `stopwords`). An empty tweet (after preprocessing) and its country label should be **excluded** from the output (`x_processed` and `y_processed`).

**Task**: Complete the `preprocess_data(data, labels)` function. The function takes **a list of tweets** and **a corresponding list of country labels** as input, and returns **two lists**. For the first list, each element is a bag-of-words representation of a tweet. For the second list, each element is a corresponding country label. Note that while we do not need to preprocess the country labels (`y`), we need to have a new output list (`y_processed`) because some tweets maybe removed after the preprocessing (due to having an empty set of bag-of-words).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [2]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

tt = TweetTokenizer()
stopwords = set(stopwords.words('english')) #note: stopwords are all in lowercase

def preprocess_data(data, labels):
    
    ###
    # Your answer BEGINS HERE
    ###
    x_processed = []
    y_processed = []
    for i in range(len(data)):
        tokens = tt.tokenize(data[i].lower())
        eng_alpha = []
        for t in tokens:
            #remove words that does not contain english alphabet
            #by removing non-alphabet characters and checking that the length of the remaining token is not 0
            alphabet = "".join(filter(str.isalpha,t))
            if len(alphabet) > 0:
                eng_alpha.append(t)
        
        #remove stopwords
        filtered_tokens = [w for w in eng_alpha if not w in stopwords]
        
        if len(filtered_tokens) > 0:
            #create a bag of words from the filtered tokens
            bag_of_words = {}
            for token in filtered_tokens:
                if token in bag_of_words:
                    bag_of_words[token] += bag_of_words[token] + 1
                else:
                    bag_of_words[token] = 1
        
            x_processed.append(bag_of_words)
            y_processed.append(labels[i])
    return x_processed, y_processed
    ###
    # Your answer ENDS HERE
    ###

x_processed, y_processed = preprocess_data(x, y)

print("Number of preprocessed tweets =", len(x_processed))
print("Number of preprocessed labels =", len(y_processed))
print("\nSamples of preprocessed data:")
for i in range(10):
    print("Country =", y_processed[i], "\tTweet =", x_processed[i])

Number of preprocessed tweets = 943
Number of preprocessed labels = 943

Samples of preprocessed data:
Country = us 	Tweet = {'@addictd2success': 1, 'thx': 1, 'u': 1, 'following': 1}
Country = us 	Tweet = {"let's": 1, 'say': 1, 'ever': 1, 'switch': 1, 'teams': 1, 'khalesi': 1, 'would': 1, 'top': 1, 'list': 1, '#girlcrush': 1}
Country = ph 	Tweet = {'taemin': 1, 'jonghyun': 1, 'birits': 1, 'make': 1, 'go': 1, 'http://t.co/le8z3dntla': 1}
Country = id 	Tweet = {'depart.senior': 1, 'rapat': 1, 'perdana': 1, 'nyayu': 1, 'anita': 1, 'others': 1, 'ruang': 1, 'aescullap': 1, 'fk': 1, 'unsri': 1, 'madang': 1, 'https://t.co/swrallnkrq': 1}
Country = ph 	Tweet = {'done': 1, 'internship': 1, 'pretty': 1, 'little': 1, 'lady': 1, 'metropolitan': 1, 'medical': 1, 'center': 1, 'w': 1, 'others': 1, 'pic': 1, 'http://t.co/1qh61r1t5r': 1}
Country = gb 	Tweet = {'wow': 1, "boruc's": 1, 'clanger': 1, 'haha': 1, 'sunday': 1, 'league': 1, 'stuff': 1, 'giroud': 1, 'believe': 1, 'luck': 1, '#clown': 1}
Countr

**For your testing**:

In [3]:
assert(len(x_processed) == len(y_processed))
assert(len(x_processed) > 800)

**Instructions**: Hashtags (i.e. topic tags which start with #) pose an interesting tokenisation problem because they often include multiple words written without spaces or capitalization. Run the code below to collect all unique hashtags in the preprocessed data. **No implementation is needed.**



In [4]:
def get_all_hashtags(data):
    hashtags = set([])
    for d in data:
        for word, frequency in d.items():
            if word.startswith("#") and len(word) > 1:
                hashtags.add(word)
    return hashtags

hashtags = get_all_hashtags(x_processed)
print("Number of hashtags =", len(hashtags))
print(sorted(hashtags))

Number of hashtags = 425
['#100percentpay', '#1stsundayofoctober', '#1yearofalmostisneverenough', '#2011prdctn', '#2015eebritishfilmacademyawards', '#2k16', '#2littlebirds', '#365picture', '#5sosacousticatlanta', '#5sosfam', '#8thannualpubcrawl', '#affsuzukicup', '#aflpowertigers', '#ahimacon14', '#aim20', '#airasia', '#allcity', '#alliswell', '#allwedoiscurls', '#amazing', '#anferneehardaway', '#ariona', '#art', '#arte', '#artwork', '#ashes', '#asian', '#asiangirl', '#askcrawford', '#askherforfback', '#askolly', '#asksteven', '#at', '#australia', '#awesome', '#awesomepict', '#barcelona', '#bart', '#bayofislands', '#beautiful', '#bedimages', '#bell', '#beringmy', '#bettybooppose', '#bff', '#big', '#bigbertha', '#bigbreakfast', '#blackhat', '#blessedmorethanicanimagine', '#blessedsunday', '#blogtourambiente', '#bluemountains', '#bonekachika', '#boomtaob', '#booyaa', '#bored', '#boredom', '#bradersisterhood', '#breaktime', '#breedingground', '#bringithomemy', '#brooksengland', '#burgers'

### Question 2 (1.0 mark)

**Instructions**: Our task here to tokenize the hashtags, by implementing the **MaxMatch algorithm** discussed in class.

NLTK has a list of words that you can use for matching, see starter code below (`words`). Be careful about efficiency with respect to doing word lookups. One extra challenge you have to deal with is that the provided list of words (`words`) includes only lemmas: your MaxMatch algorithm should match inflected forms by converting them into lemmas using the NLTK lemmatizer before matching (provided by the function `lemmatize(word)`). Note that the list of words (`words`) is the only source that you'll use for matching (i.e. you do not need to find  other external word lists). If you are unable to make any longer match, your code should default to matching a single letter.

For example, given "#newrecord", the algorithm should produce: \["#", "new", "record"\].

**Task**: Complete the `tokenize_hashtags(hashtags)` function by implementing the MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of tokenised words".

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [5]:
from nltk.corpus import wordnet

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
words = set(nltk.corpus.words.words()) #a list of words provided by NLTK
words = set([ word.lower() for word in words ]) #lowercase all the words for better matching

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


def tokenize_hashtags(hashtags):
    ###
    # Your answer BEGINS HERE
    ###
    tags_dict = {}
    for tags in hashtags:
        tokens = []
        i = 0
        while i < len(tags):
            max_word = tags[i]
            for j in range(len(tags),i,-1):
                tmp_word = tags[i:j]
                lemma = lemmatize(tmp_word)
                if lemma in words and len(tmp_word) > len(max_word):
                    max_word = tmp_word
                    break
            i += len(max_word)
            tokens.append(max_word)
        tags_dict[tags] = tokens
    return tags_dict
    ###
    # Your answer ENDS HERE
    ###

#tokenise hashtags with MaxMatch
tokenized_hashtags = tokenize_hashtags(hashtags)

#print results
for k, v in sorted(tokenized_hashtags.items())[-30:]:
    print(k, v)

#vanilla ['#', 'vanilla']
#vca ['#', 'v', 'ca']
#vegan ['#', 'vega', 'n']
#veganfood ['#', 'vega', 'n', 'food']
#vegetables ['#', 'vegetables']
#vegetarian ['#', 'vegetarian']
#video ['#', 'video']
#vma ['#', 'v', 'ma']
#voteonedirection ['#', 'vote', 'one', 'direction']
#vsco ['#', 'vs', 'c', 'o']
#vscocam ['#', 'vs', 'coca', 'm']
#walking ['#', 'walking']
#watch ['#', 'watch']
#weare90s ['#', 'wear', 'e', '9', '0', 's']
#wearesocial ['#', 'weares', 'o', 'c', 'i', 'al']
#white ['#', 'white']
#wings ['#', 'wings']
#wok ['#', 'wo', 'k']
#wood ['#', 'wood']
#work ['#', 'work']
#workmates ['#', 'work', 'mates']
#world ['#', 'world']
#worldcup2014 ['#', 'world', 'cup', '2', '0', '1', '4']
#yellow ['#', 'yellow']
#yiamas ['#', 'y', 'i', 'ama', 's']
#ynwa ['#', 'yn', 'wa']
#youtube ['#', 'you', 'tube']
#yummy ['#', 'yummy']
#yws13 ['#', 'y', 'ws', '1', '3']
#zweihandvollfarm ['#', 'z', 'wei', 'hand', 'vol', 'l', 'farm']


**For your testing:**

In [6]:
assert(len(tokenized_hashtags) == len(hashtags))
assert(tokenized_hashtags["#newrecord"] == ["#", "new", "record"])

### Question 3 (1.0 mark)

**Instructions**: Our next task is to tokenize the hashtags again, but this time using a **reversed version of the MaxMatch algorithm**, where matching begins at the end of the hashtag and progresses backwards (e.g. for <i>#helloworld</i>, we would process it right to left, starting from the last character <i>d</i>). Just like before, you should use the provided word list (`words`) for word matching.

**Task**: Complete the `tokenize_hashtags_rev(hashtags)` function by the MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of tokenised words".

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [7]:
def tokenize_hashtags_rev(hashtags):
    ###
    # Your answer BEGINS HERE
    ###
    rev_tags_dict = {}
    for tags in hashtags:
        tokens = []
        i = len(tags) - 1
        while i >= 0:
            max_word = tags[i]
            for j in range(0,i):
                tmp_word = tags[j:i+1]
                lemma = lemmatize(tmp_word)
                if lemma in words and len(tmp_word) > len(max_word):
                    max_word = tmp_word
                    break
            i -= len(max_word)
            tokens = [max_word] + tokens
        rev_tags_dict[tags] = tokens
    return rev_tags_dict
    ###
    # Your answer ENDS HERE
    ###

    
#tokenise hashtags with the reversed version of MaxMatch
tokenized_hashtags_rev = tokenize_hashtags_rev(hashtags)

#print results
for k, v in sorted(tokenized_hashtags_rev.items())[-30:]:
    print(k, v)

#vanilla ['#', 'vanilla']
#vca ['#', 'v', 'ca']
#vegan ['#', 'v', 'e', 'gan']
#veganfood ['#', 'v', 'e', 'gan', 'food']
#vegetables ['#', 'vegetables']
#vegetarian ['#', 'vegetarian']
#video ['#', 'video']
#vma ['#', 'v', 'ma']
#voteonedirection ['#', 'vote', 'one', 'direction']
#vsco ['#', 'vs', 'c', 'o']
#vscocam ['#', 'vs', 'c', 'o', 'cam']
#walking ['#', 'walking']
#watch ['#', 'watch']
#weare90s ['#', 'we', 'are', '9', '0', 's']
#wearesocial ['#', 'we', 'are', 'social']
#white ['#', 'white']
#wings ['#', 'wings']
#wok ['#', 'w', 'ok']
#wood ['#', 'wood']
#work ['#', 'work']
#workmates ['#', 'work', 'mates']
#world ['#', 'world']
#worldcup2014 ['#', 'world', 'cup', '2', '0', '1', '4']
#yellow ['#', 'yellow']
#yiamas ['#', 'y', 'i', 'a', 'mas']
#ynwa ['#', 'yn', 'wa']
#youtube ['#', 'you', 'tube']
#yummy ['#', 'yummy']
#yws13 ['#', 'y', 'ws', '1', '3']
#zweihandvollfarm ['#', 'z', 'wei', 'hand', 'vol', 'l', 'farm']


**For your testing:**

In [8]:
assert(len(tokenized_hashtags_rev) == len(hashtags))
assert(tokenized_hashtags_rev["#newrecord"] == ["#", "new", "record"])

### Question 4 (1.0 mark)

**Instructions**: The two versions of MaxMatch will produce different results for some of the hashtags. For a hastag that has different results, our task here is to use a **unigram language model** (lecture 3) to score them to see which is better. Recall that in a unigram language model we compute P(<i>#</i>, <i>hello</i>, <i>world</i> = P(<i>#</i>)\*P(<i>hellow</i>)\*P(<i>world</i>).

You should: (1) use the NLTK's Brown corpus (`brown_words`) for collecting word frequencies (note: the words are already tokenised so no further tokenisation is needed); (2) lowercase all words in the corpus; (3) use add-one smoothing when computing the unigram probabilities; and (4) work in the log space to prevent numerical underflow.

**Task**: Build a unigram language model with add-one smoothing using the word counts from the Brown corpus. Iterate through the hashtags, and for each hashtag where MaxMatch and reversed MaxMatch produce different results, print the following: (1) the hashtag; (2) the results produced by MaxMatch and reversed MaxMatch; and (3) the log probability of each result as given by the unigram language model. Note: you **do not** need to print the hashtags where MaxMatch and reversed MaxMatch produce the same results.

An example output:
```
1. #abcd
MaxMatch = [#, a, bc, d]; LogProb = -2.3
Reversed MaxMatch = [#, a, b, cd]; LogProb = -3.5

2. #efgh
MaxMatch = [#, ef, g, h]; LogProb = -4.2
Reversed MaxMatch = [#, e, fgh]; LogProb = -3.1

```

Have a look at the output, and see if the sequences with better language model scores (i.e. less negative) are generally more coherent.

In [9]:
from nltk.corpus import brown
import numpy as np

#words from brown corpus
brown_words = brown.words()

###
# Your answer BEGINS HERE
###
#Build the word frequency dictionary
brown_dict = {}
total_tokens = len(brown_words)
for words in brown_words:
    if words.lower() in brown_dict:
        brown_dict[words.lower()] += 1
    else:
        brown_dict[words.lower()] = 1
total_types = len(brown_dict)

count = 0 #count number of different results
#Iterate through the hashtags
for hashtag in tokenized_hashtags:
    #MaxMatch and reversed MaxMatch produce different results
    if tokenized_hashtags[hashtag] != tokenized_hashtags_rev[hashtag]:
        count += 1
        print(count,".",hashtag)
        
        #MaxMatch
        prob_maxMatch = 0
        for token in tokenized_hashtags[hashtag]:
            if token in brown_dict:
                prob_maxMatch += np.log((brown_dict[token] + 1) / (total_tokens + total_types))
            else:
                prob_maxMatch += np.log(1 / (total_tokens + total_types))
        print("MaxMatch =",tokenized_hashtags[hashtag],"; LogProb =",prob_maxMatch)
        
        #reversed MaxMatch
        prob_revMaxMatch = 0
        for token in tokenized_hashtags_rev[hashtag]:
            if token in brown_dict:
                prob_revMaxMatch += np.log((brown_dict[token] + 1) / (total_tokens + total_types))
            else:
                prob_revMaxMatch += np.log(1 / (total_tokens + total_types))
        print("Reversed MaxMatch =",tokenized_hashtags_rev[hashtag],"; LogProb =",prob_revMaxMatch,"\n")
###
# Your answer ENDS HERE
###

1 . #anferneehardaway
MaxMatch = ['#', 'an', 'fern', 'e', 'eh', 'ar', 'daw', 'ay'] ; LogProb = -97.22465036397413
Reversed MaxMatch = ['#', 'an', 'f', 'er', 'nee', 'hard', 'away'] ; LogProb = -74.77282410312905 

2 . #bringithomemy
MaxMatch = ['#', 'bring', 'it', 'home', 'my'] ; LogProb = -42.3969400397748
Reversed MaxMatch = ['#', 'brin', 'git', 'home', 'my'] ; LogProb = -54.934471662227295 

3 . #southampton
MaxMatch = ['#', 'south', 'am', 'p', 'ton'] ; LogProb = -52.09488365569565
Reversed MaxMatch = ['#', 's', 'out', 'ham', 'p', 'ton'] ; LogProb = -63.278947787283734 

4 . #bradersisterhood
MaxMatch = ['#', 'brad', 'ers', 'ist', 'er', 'hood'] ; LogProb = -80.57604091439234
Reversed MaxMatch = ['#', 'brad', 'er', 'sisterhood'] ; LogProb = -55.334704030901435 

5 . #insightmediasingapore
MaxMatch = ['#', 'insight', 'media', 'sing', 'a', 'pore'] ; LogProb = -63.5615297924938
Reversed MaxMatch = ['#', 'insight', 'me', 'di', 'as', 'inga', 'pore'] ; LogProb = -74.71486738172212 

6 . #th

# Text Classification (4 marks)

### Question 5 (1.0 mark)

**Instructions**: Here we are interested to do text classification, to predict the country of origin of a given tweet. The task here is to create training, development and test partitions from the preprocessed data (`x_processed`) and convert the bag-of-words representation into feature vectors.

**Task**: Create training, development and test partitions with a 70%/15%/15% ratio. Remember to preserve the ratio of the classes for all your partitions. That is, say we have only 2 classes and 70% of instances are labelled class A and 30% of instances are labelled class B, then the instances in training, development and test partitions should also preserve this 7:3 ratio. You may use sklearn's builtin functions for doing data partitioning.

Next, turn the bag-of-words dictionary of each tweet into a feature vector. You may also use sklearn's builtin functions for doing this.

You should produce 6 objects: `x_train`, `x_dev`, `x_test` which contain the input feature vectors, and `y_train`, `y_dev` and `y_test` which contain the labels.

In [10]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

x_train, x_dev, x_test = None, None, None
y_train, y_dev, y_test = None, None, None

###
# Your answer BEGINS HERE
###
def split_data(x_data, y_data):
    #split once to 70% train set and 30% validation-test set
    x_train, x_dev_test, y_train, y_dev_test = train_test_split(x_data, y_data, 
                                                                train_size = 0.7, test_size = 0.3, 
                                                                random_state = 42, stratify = y_data)
    #split 30% validation-test set further to 50% validation set (30%*50% = 15% validation set)
    #and 50% test set (30%*50% = 15% test set)
    x_dev, x_test, y_dev, y_test = train_test_split(x_dev_test, y_dev_test, 
                                                    train_size = 0.5, test_size = 0.5, 
                                                    random_state = 42, stratify = y_dev_test)
    
    return x_train, x_dev, x_test, y_train, y_dev, y_test    

x_train, x_dev, x_test, y_train, y_dev, y_test = split_data(x_processed, y_processed)
    
#transform the bag-of-words to feature vectors using built-in function DictVectorizer()
vectorizer = DictVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_dev = vectorizer.transform(x_dev)
x_test = vectorizer.transform(x_test)
    
#uncomment the following lines to transform the class labels if necessary
#le = preprocessing.LabelEncoder()
#y_train = le.fit_transform(y_train)
#y_dev = le.transform(y_dev)
#y_test = le.transform(y_test)

###
# Your answer ENDS HERE
###

### Question 6 (1.0 mark)

**Instructions**: Now, let's build some classifiers. Here, we'll be comparing Naive Bayes and Logistic Regression. For each, you need to first find a good value for their main regularisation hyper-parameters, which you should identify using the scikit-learn docs or other resources. Use the development set you created for this tuning process; do **not** use cross-validation in the training set, or involve the test set in any way. You don't need to show all your work, but you do need to print out the **accuracy** with enough different settings to strongly suggest you have found an optimal or near-optimal choice. We should not need to look at your code to interpret the output.

**Task**: Implement two text classifiers: Naive Bayes and Logistic Regression. Tune the hyper-parameters of these classifiers and print the task performance (accuracy) for different hyper-parameter settings.

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

###
# Your answer BEGINS HERE
###
#MultinomialNB hyper-parameter tuning
alpha = np.linspace(0.00001, 1, 20)
best_score_mnb = 0
best_alpha = 0
print("Parameter tuning and accuracy scores for Naive Bayes\n")
for a in alpha:
    mnb = MultinomialNB(alpha=a)
    mnb.fit(x_train, y_train)
    y_pred_mnb = mnb.predict(x_dev)
    accuracy_mnb = metrics.accuracy_score(y_dev, y_pred_mnb)
    print("Naive Bayes accuracy:", accuracy_mnb, "with alpha=", a)
    if accuracy_mnb > best_score_mnb:
        best_score_mnb = accuracy_mnb
        best_alpha = a
print("\nBest Score: ", best_score_mnb, "Best alpha: ", best_alpha,"\n")
        
print("===================================================\n")

#Logistic Regression hyper-parameter tuning   
C = np.linspace(0.01, 1, 10)
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
best_score_lr = 0
best_C = 0
best_solver = None
print("Parameter tuning and accuracy scores for Logistic Regression\n")
for c in C:
    for s in solver:
        #set max_iter to 1000 to avoid non-convergence warnings
        #set random_state used by sag, saga, and liblinear to shuffle data
        lr = LogisticRegression(C=c, solver=s, max_iter=1000, random_state=42)
        lr.fit(x_train, y_train)
        y_pred_lr = lr.predict(x_dev)
        accuracy_lr = metrics.accuracy_score(y_dev, y_pred_lr)
        print("Logistic Regression accuracy:", accuracy_lr, "with C=", c, ", solver=", s)
        if accuracy_lr > best_score_lr:
            best_score_lr = accuracy_lr
            best_C = c
            best_solver = s
print("\nBest Score: ", best_score_lr, "Best parameters: C=", best_C,", solver=", best_solver)
###
# Your answer ENDS HERE
###

Parameter tuning and accuracy scores for Naive Bayes

Naive Bayes accuracy: 0.2978723404255319 with alpha= 1e-05
Naive Bayes accuracy: 0.2978723404255319 with alpha= 0.05264105263157895
Naive Bayes accuracy: 0.2695035460992908 with alpha= 0.10527210526315789
Naive Bayes accuracy: 0.2624113475177305 with alpha= 0.15790315789473686
Naive Bayes accuracy: 0.2624113475177305 with alpha= 0.2105342105263158
Naive Bayes accuracy: 0.2624113475177305 with alpha= 0.26316526315789474
Naive Bayes accuracy: 0.2624113475177305 with alpha= 0.3157963157894737
Naive Bayes accuracy: 0.2624113475177305 with alpha= 0.3684273684210526
Naive Bayes accuracy: 0.2624113475177305 with alpha= 0.4210584210526316
Naive Bayes accuracy: 0.2553191489361702 with alpha= 0.47368947368421055
Naive Bayes accuracy: 0.2553191489361702 with alpha= 0.5263205263157894
Naive Bayes accuracy: 0.2553191489361702 with alpha= 0.5789515789473684
Naive Bayes accuracy: 0.2624113475177305 with alpha= 0.6315826315789473
Naive Bayes accura

### Question 7 (1.0 mark)

**Instructions**: Using the best settings you have found, compare the two classifiers based on performance in the test set. Print out both **accuracy** and **macro-averaged F-score** for each classifier. Be sure to label your output. You may use sklearn's inbuilt functions.

**Task**: Compute test performance in terms of accuracy and macro-averaged F-score for both Naive Bayes and Logistic Regression, using their optimal hyper-parameter settings based on their development performance.

In [12]:
###
# Your answer BEGINS HERE
###
#Naive Bayes performance scores using best setting where alpha=0.00001
mnb = MultinomialNB(alpha=0.00001)
mnb.fit(x_train, y_train)
y_pred_mnb = mnb.predict(x_test)
mnb_accuracy = metrics.accuracy_score(y_test, y_pred_mnb)
mnb_macro_fscore = metrics.f1_score(y_test, y_pred_mnb, average='macro')
print("Naive Bayes accuracy:", mnb_accuracy, "with alpha=0.00001")
print("Naive Bayes macro-averaged F-score:", mnb_macro_fscore, "with alpha=0.00001\n")

print("===================================================\n")

#Logistic Regression performance scores using best setting where C=0.23 and solver='newton-cg'
lr = LogisticRegression(C=0.23, solver='newton-cg')
lr.fit(x_train, y_train)
y_pred_lr = lr.predict(x_test)
lr_accuracy = metrics.accuracy_score(y_test, y_pred_lr)
lr_macro_fscore = metrics.f1_score(y_test, y_pred_lr, average='macro')
print("Logistic Regression accuracy:", lr_accuracy, "with C=0.23 and solver='newton-cg'")
print("Logistic Regression macro-averaged F-score:", lr_macro_fscore, "with C=0.23 and solver='newton-cg'")
###
# Your answer ENDS HERE
###

Naive Bayes accuracy: 0.28169014084507044 with alpha=0.00001
Naive Bayes macro-averaged F-score: 0.27290219972333224 with alpha=0.00001


Logistic Regression accuracy: 0.2887323943661972 with C=0.23 and solver='newton-cg'
Logistic Regression macro-averaged F-score: 0.27044621782623357 with C=0.23 and solver='newton-cg'


### Question 8 (1.0 mark)

**Instructions**: Print the most important features and their weights for each class for the two classifiers.


**Task**: For each of the classifiers (Logistic Regression and Naive Bayes) you've built in the previous question, print out the top-20 features (words) with the highest weight for each class (countries).

An example output:
```
Classifier = Logistic Regression

Country = au
aaa (0.999) bbb (0.888) ccc (0.777) ...

Country = ca
aaa (0.999) bbb (0.888) ccc (0.777) ...

Classifier = Naive Bayes

Country = au
aaa (-1.0) bbb (-2.0) ccc (-3.0) ...

Country = ca
aaa (-1.0) bbb (-2.0) ccc (-3.0) ...
```

Have a look at the output, and see if you notice any trend/pattern in the words for each country.

In [13]:
###
# Your answer BEGINS HERE
###
feature_names = vectorizer.get_feature_names()
#Top 20 words for each country using Naive Bayes
print("Classifier = Naive Bayes\n")
for c in range(len(mnb.classes_)):
    print("Country =",mnb.classes_[c])
    
    #label the weights with feature names (corresponding words)
    #then sort the weights (log probability) from largest to smallest
    coefs_fns = sorted(zip(mnb.feature_log_prob_[c], feature_names), reverse=True)
    
    #extract the top 20-features (words) for each class (country)
    top20 = ""
    for coef, fn in coefs_fns[:20]:
        top20 = top20 + fn + " (" + str(round(coef,5)) + ") "
    print(top20,"\n")

print("===================================================\n")

#Top 20 words for each country using Logistic Regression
print(("Classifier = Logistic Regression\n"))
for c in range(len(lr.classes_)):
    print("Country =",lr.classes_[c])
    
    #label the weights with feature names (corresponding words)
    #then sort the weights (coefficient) from largest to smallest
    coefs_fns = sorted(zip(lr.coef_[c], feature_names), reverse=True)
    
    #extract the top 20-features (words) for each class (country)
    top20 = ""
    for coef, fn in coefs_fns[:20]:
        top20 = top20 + fn + " (" + str(round(coef,5)) + ") "
    print(top20,"\n")
###
# Your answer ENDS HERE
###

Classifier = Naive Bayes

Country = au
melbourne (-4.04836) one (-4.7415) i'm (-4.7415) great (-4.7415) little (-4.96465) keep (-4.96465) australia (-4.96465) win (-5.25233) vca (-5.25233) two (-5.25233) tomorrow (-5.25233) team (-5.25233) sledging (-5.25233) make (-5.25233) love (-5.25233) even (-5.25233) cry (-5.25233) come (-5.25233) away (-5.25233) anything (-5.25233)  

Country = ca
maybe (-3.60589) i'm (-4.76904) first (-4.76904) u (-4.99218) school (-4.99218) right (-4.99218) like (-4.99218) great (-4.99218) think (-5.27987) thing (-5.27987) thanks (-5.27987) story (-5.27987) minute (-5.27987) learning (-5.27987) hate (-5.27987) got (-5.27987) good (-5.27987) dun (-5.27987) class (-5.27987) big (-5.27987)  

Country = de
i'm (-4.20481) year (-4.49249) week (-4.49249) posted (-4.49249) painting (-4.49249) night (-4.49249) remstal (-4.89796) photodesign (-4.89796) photo (-4.89796) mkmedi (-4.89796) miss (-4.89796) kernen (-4.89796) im (-4.89796) good (-4.89796) enough (-4.89796) d